# LlamaParse with GPT-4o


<a href="https://colab.research.google.com/github/run-llama/llama_parse/blob/main/examples/demo_gpt4o/gpt4o_tesla_impact_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPT-4o is a [fully multimodal model by OpenAI](https://openai.com/index/hello-gpt-4o/) released in May 2024. It matches GPT-4 Turbo performance in text and code, and has significantly improved vision and audio capabilities.

The expanded vision/audio capabilities mean that it can be used for document parsing, by treating each page as an image and performing document extraction. We support using GPT-4o natively in LlamaParse for document parsing. The notebook below walks you through an example of using GPT-4o over the Tesla impact report.

**NOTE**: The pricing for LlamaParse + gpt4o is an order more expensive than using LlamaParse by default. Currently, every page parsed with gpt4o counts for 200 pages in the LlamaParse usage tracker.


In [12]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
import os

In [ ]:
os.environ["LLAMA_CLOUD_API_KEY"] = "<LLAMA_CLOUD_API_KEY>

### Use LlamaParse with `gpt4o_mode=True`

By turning on gpt4o, we use GPT-4o multimodal capabilities to do document parsing per page instead of the LlamaParse default pipeline.

We load a snippet of the [2019 Tesla impact report](https://www.tesla.com/ns_videos/2019-tesla-impact-report.pdf). To help you save tokens, we only load 4 pages of this report (which will count as 800 pages of LlamaParse pages). 

You can optionally choose to provide a `gpt4o_api_key`. If you do this, then we will use your API key to make GPT-4o calls, and your LlamaParse usage will be counted as if `gpt4o_mode` was not turned on (each page will be counted as a page instead of 200 pages). 

In [22]:
from llama_parse import LlamaParse

parser_gpt4o = LlamaParse(
    result_type="markdown",
    base_url="https://api.staging.llamaindex.ai",
    api_key=api_key,
    gpt4o_mode=True,
    # gpt4o_api_key="<gpt4o_api_key>"
)

In [23]:
documents_gpt4o = parser_gpt4o.load_data('./2019-tesla-impact-report-short.pdf')

Started parsing the file under job_id 03b089a3-0f1b-40e9-ab73-67fa430fb329
.

In [24]:
print(documents_gpt4o[0].get_content())

# Impact Report
## 2019

![Tesla Logo](https://www.tesla.com/sites/default/files/blog_images/tesla_announcement_social.jpg)

![Earth and Car in Space](https://www.tesla.com/sites/default/files/blog_images/tesla_announcement_social.jpg)
---
# Introduction 03

# Mission and Tesla Ecosystem 04

# Environmental Impact 06
- Lifecycle Analysis of Tesla Vehicles versus Average ICE
- Battery Recycling
- NOx, Particulates and Other Pollutants
- Water Used per Vehicle Manufactured
- Emissions Credits
- Net Energy Impact of Our Products

# Product Impact 20
- Price Equivalency
- Primary Driver
- Long Distance Travel
- Active Safety
- Passive Safety
- Tesla Safety Awards
- Fire Safety
- Cyber Security
- Disaster Relief
- Resilience of the Grid
- Megapack
- Solar Roof

# Supply Chain 33
- Responsible Material Sourcing
- Cobalt Sourcing

# People and Culture 37
- Our Environmental, Health, and Safety Strategy
- Safety Improvements
- Case Study: Ergonomics and Model Y Design
- Rewarding the Individua

## Build RAG pipeline over the Parsed Report

We now try building a RAG pipeline over this parsed report. It's not a lot of text, but we split it into chunks and load it into a simple in-memory vector store.

We ask a question over the parsed markdown table and get back the right answer! We also ask a question over the text.

In [25]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter


splitter = SentenceSplitter()
nodes = splitter.get_nodes_from_documents(documents_gpt4o)

In [26]:
vector_index = VectorStoreIndex(nodes)

In [27]:
query_engine = vector_index.as_query_engine()

In [28]:
response = query_engine.query("What are the greenhouse emissions for agriculture and transportation?") 

In [29]:
print(str(response))

Agriculture accounts for 20% of global greenhouse gas emissions, while transportation contributes 16% to these emissions.


Let's also try asking a question over another piece of the text.

In [30]:
response = query_engine.query("What does Tesla aim to do?") 

In [31]:
print(str(response))

Tesla aims to accelerate the world's transition to sustainable energy by creating a complete energy and transportation ecosystem that includes solar generation, energy storage, and all-electric vehicles with zero tailpipe emissions.
